# DeepState - Experiments

- Using DeepState with gluonts to run some experiments
- Follow this installation guide: http://gluon-ts-staging.s3-accelerate.dualstack.amazonaws.com/PR-271/4/install.html
- Important note! DeepState, DF-RNN are only released for versions >="0.4.*"
- For development version, follow **Install from Source Code** (which I did) >> **Install from Github**

### Deep State Space Models (Rangapuram et al., 2018)

**DeepState** combines state space models (SSM) with a recurrent neural network (RNN). The SSM is applied *locally* to the individual time series that is parametrized using a *global* representation in form of a RNN. The RNN is trained on the entire dataset. 

- [`DeepStateEstimator`](http://gluon-ts.mxnet.io/master/api/gluonts/gluonts.model.deepstate.html#gluonts.model.deepstate.DeepStateEstimator)


In [1]:
!pip install mxnet=="1.4.1"

    100% |████████████████████████████████| 28.4MB 1.9MB/s  eta 0:00:01
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
# !pip install gluonts=="0.4.1"

In [2]:
!pip install gluonts

    100% |████████████████████████████████| 327kB 49.2MB/s ta 0:00:01
    100% |████████████████████████████████| 194kB 62.4MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/41/17/c62faccbfbd163c7f57f3844689e3a78bae1f403648a6afb1d0866d87fbb/python_dateutil-2.8.0-py2.py3-none-any.whl
    100% |████████████████████████████████| 61kB 44.4MB/s ta 0:00:01
    100% |████████████████████████████████| 10.4MB 5.3MB/s  eta 0:00:01
    100% |████████████████████████████████| 6.9MB 7.4MB/s eta 0:00:011
    100% |████████████████████████████████| 81kB 53.0MB/s ta 0:00:01
  Running setup.py bdist_wheel for ujson ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/28/77/e4/0311145b9c2e2f01470e744855131f9e34d6919687550f87d1
  Running setup.py bdist_wheel for holidays ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/40/a9/2b/94ac5464363d37564a87dc93a9d21a5850aac14a4608197003
Successfully built ujson holidays
  Found existing installation: python-dateut

In [5]:
import gluonts
gluonts.__version__

'0.4.2'

In [6]:
import mxnet
mxnet.__version__

'1.4.1'

In [7]:
import pprint

# standard imports
import numpy as np
import pandas as pd

# vis imports 
%matplotlib inline
import matplotlib.pyplot as plt

# json
import json

# gluon data 
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# model imports 
from gluonts.model.deepstate import DeepStateEstimator
# from gluonts.model.deep_factor import DeepFactorEstimator
# from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint

INFO:root:Using CPU


In [10]:
def deep_state(seed=42, data="m4_quarterly", epochs=100, batches=50):
    
    mx.random.seed(seed)
    np.random.seed(seed)
    
    dataset = get_dataset(data)

    trainer = Trainer(
        ctx=mx.cpu(0),
#         ctx=mx.gpu(0),
        epochs=epochs,
        num_batches_per_epoch=batches,
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepStateEstimator(
        trainer=trainer,
        cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True,

    )

    predictor = estimator.train(dataset.train)
    
#     predictor = estimator.train(training_data=dataset.train, 
#                                 validation_data=dataset.test)
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}
    output["epochs"] = epochs
    output["seed"] = seed
    
    df=pd.DataFrame([output])
    pprint(output)
    
    return df

In [11]:
%%time
if __name__ == "__main__":
    df = deep_state(seed=42, data="m4_weekly", epochs=2, batches=2)
df

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_weekly.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 2/2 [00:07<00:00,  3.71s/it, avg_epoch_loss=0.606]
INFO:root:Epoch[0] Elapsed time 7.426 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=0.605787
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 2/2 [00:04<00:00,  2.46s/it, avg_epoch_loss=0.549]
INFO:root:Epoch[1] Elapsed time 4.913 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=0.548566
INFO:root:Loading parameters from best epoch (1)
INFO:root:Final loss: 0.5485656559467316 (occurred at epoch 1)
INFO:root:End model training
Running evaluation: 100%|██████████| 359/359 [00:49<00:00,  7.26it/s]

{'MASE': 11.89050593,
 'MSIS': 133.21872492,
 'epochs': 2,
 'sMAPE': 0.16463858,
 'seed': 42,
 'wQuantileLoss[0.5]': 0.14957976,
 'wQuantileLoss[0.9]': 0.1158686}
CPU times: user 1min 55s, sys: 10.7 s, total: 2min 5s
Wall time: 1min 2s


In [12]:
df

,MASE,sMAPE,MSIS,wQuantileLoss[0.5],wQuantileLoss[0.9],epochs,seed
0,11.890506,0.164639,133.218725,0.14958,0.115869,2,42


In [ ]:
!jupyter nbconvert --output-dir="./html_outputs" --to html deep_state01-deep_state_experiments_with_gluonts.ipynb

[NbConvertApp] Converting notebook DeepState01-DeepState_experiments_with_gluonts.ipynb to html
[NbConvertApp] Writing 1312413 bytes to ./html_outputs/DeepState01-DeepState_experiments_with_gluonts.html


In [7]:
# %%time
# results = []

# if __name__ == "__main__":
#     for i in range(42, 45):
#         print("Seed:", i)
#         res = deep_state(data="m4_weekly", seed=i, epochs=1, batches=1)
#         pprint(res)
#         results.append(res)
    
# results